In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, k_means
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.feature_selection import SelectKBest
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../data/mlb-pitches.csv')
df.drop(columns = ['Unnamed: 0'], inplace = True) 

pd.set_option('max_columns', None)

print(df.shape)
df.head(3)
#list(df.columns.values)
#df.info()

(708653, 76)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,post_away_score,post_home_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bauer_units,is_strike,pitch_count,swing_miss,plate_-x,pfx_-x
0,FF,2021-10-03,92.3,1.40,6.80,"Smith, Will",596019,519293,out,hit_into_play,1.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,X,5.0,ground_ball,1,2,2021,8.28,16.56,-0.69,2.83,NaN,NaN,NaN,2,9,Top,77.54,155.31,-6.833043,-134.166485,-7.361843,9.708393,26.562803,-14.083224,3.32,1.51,13.0,95.2,-13.0,92.8,2330.0,6.5,632254,54.03,0.174,0.158,0.0,1.0,0.0,0.0,2.0,61,4,4-Seam Fastball,5,0,0,5,Infield shift,Strategic,148.0,0.0,-0.073,25.243770,1,1-2,0,0.69,-8.28
1,SL,2021-10-03,80.6,1.60,6.64,"Smith, Will",596019,519293,NaN,foul,4.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,NaN,NaN,1,1,2021,-9.24,5.76,-0.71,2.62,NaN,NaN,NaN,2,9,Top,NaN,NaN,-3.700232,-117.430885,-3.266842,-6.531123,19.793390,-27.369114,3.32,1.51,108.0,75.3,75.0,81.2,2254.0,6.4,632254,54.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,Slider,5,0,0,5,Infield shift,Strategic,315.0,0.0,-0.027,27.965261,1,1-1,0,0.71,9.24
2,CU,2021-10-03,75.5,1.46,6.88,"Smith, Will",596019,519293,NaN,foul,5.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,NaN,NaN,1,0,2021,-7.80,-6.12,-0.04,2.46,NaN,NaN,NaN,2,9,Top,NaN,NaN,-1.977183,-109.901781,-1.155694,-4.872924,20.602334,-36.262184,3.32,1.51,157.0,83.5,65.0,75.2,1940.0,6.2,632254,54.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,Curveball,5,0,0,5,Infield shift,Standard,328.0,0.0,-0.020,25.695364,1,1-0,0,0.04,7.80


In [3]:
#df[df.columns[df.isnull().any()]].isnull().sum()
#df.dropna(inplace = True)
df.dropna(subset = ['release_extension', 'release_spin_rate', 'if_fielding_alignment'], 
          inplace = True)

# Fill nulls with none so they aren't dropped
df['bb_type'] = df['bb_type'].fillna('not_in_play').astype(str)
df['events'] = df['events'].fillna('none').astype(str)
df.fillna(value = 0, inplace = True)


df = pd.get_dummies(df, columns = ['p_throws', 'type', 'stand', 'events', 
                                   'description', 'bb_type', 'if_fielding_alignment', 
                                   'of_fielding_alignment'])

# Drop pitch types Fasball, knuckleball, eephus, and screwball
df['pitch_type'] = df['pitch_type'].map({'FS': 8, 'CH': 7, 'KC': 6, 'CU': 5,
                                         'SL': 4, 'FC': 3, 'SI': 2, 'FF': 1})

In [4]:
pitch_data = df[['pitch_type', 'release_speed', 'release_spin_rate', 'spin_axis', 
                 'pfx_-x', 'pfx_z','release_extension', 'release_pos_x', 'release_pos_z',
                 'p_throws_L', 'p_throws_R', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
                 'woba_value', 'babip_value', 'iso_value', 'launch_speed_angle', 'hit_distance_sc', 
                 'launch_speed', 'launch_angle']]

pitch_data.dropna(inplace = True)

ff = pitch_data.loc[pitch_data['pitch_type'] == 1]
si = pitch_data.loc[pitch_data['pitch_type'] == 2]
fc = pitch_data.loc[pitch_data['pitch_type'] == 3]
sl = pitch_data.loc[pitch_data['pitch_type'] == 4]
cu = pitch_data.loc[pitch_data['pitch_type'] == 5]
kc = pitch_data.loc[pitch_data['pitch_type'] == 6]
ch = pitch_data.loc[pitch_data['pitch_type'] == 7]
fs = pitch_data.loc[pitch_data['pitch_type'] == 8]


print(pitch_data.shape)
pitch_data.head()

(702437, 20)


,pitch_type,release_speed,release_spin_rate,spin_axis,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z,p_throws_L,p_throws_R,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,babip_value,iso_value,launch_speed_angle,hit_distance_sc,launch_speed,launch_angle
0,1,92.3,2330.0,148.0,-8.28,16.56,6.5,1.40,6.80,1,0,0.174,0.158,0.0,0.0,0.0,2.0,13.0,95.2,-13.0
1,4,80.6,2254.0,315.0,9.24,5.76,6.4,1.60,6.64,1,0,0.000,0.000,0.0,0.0,0.0,0.0,108.0,75.3,75.0
2,5,75.5,1940.0,328.0,7.80,-6.12,6.2,1.46,6.88,1,0,0.000,0.000,0.0,0.0,0.0,0.0,157.0,83.5,65.0
3,5,75.0,2017.0,330.0,8.28,-8.28,5.9,1.53,6.83,1,0,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,91.2,2281.0,143.0,-7.56,15.36,6.3,1.49,6.66,1,0,0.100,0.090,0.0,0.0,0.0,2.0,9.0,93.3,-18.0


In [5]:
X = ff

ss = StandardScaler()
X_scaled = ss.fit_transform(X)
X_scaled[:,:] *= -1

pca = PCA().fit(X_scaled)
principal_components = pca.transform(X_scaled)

model = pd.DataFrame(data = principal_components, columns = X.columns)

print(model.shape)
model.head()

(248739, 20)


,pitch_type,release_speed,release_spin_rate,spin_axis,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z,p_throws_L,p_throws_R,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,babip_value,iso_value,launch_speed_angle,hit_distance_sc,launch_speed,launch_angle
0,0.785934,3.322322,-0.156211,-0.848082,0.220316,0.438920,-0.757524,0.013429,1.602032,-1.200417,1.028757,0.200847,0.421802,0.848606,-0.435296,-0.052425,-0.150232,0.000260,1.586190e-16,0.0
1,0.441210,3.422466,0.172852,-0.301125,0.592863,0.135734,-0.702195,0.082042,1.451826,-1.210711,1.334123,0.220045,0.639001,0.670121,-0.317567,-0.080178,0.063214,0.074104,1.314322e-16,0.0
2,-1.189246,3.221710,0.053528,-1.148478,0.490466,0.144417,0.015146,0.021698,0.106394,-0.726307,-0.214431,0.330135,-0.018579,-0.082633,-0.363902,-0.050971,0.048578,-0.000167,2.815175e-16,0.0
3,-1.192971,3.364016,0.093868,-1.065089,0.446772,0.621207,0.032355,0.247348,0.121342,-0.840340,-0.229140,0.125701,-0.022591,-0.084403,-0.389946,-0.054112,-0.120463,-0.000223,2.815175e-16,0.0
4,-0.992384,-1.750959,1.381213,1.348142,0.817382,0.217161,-0.009009,-1.060932,-0.031567,-0.391313,-0.286378,0.328148,-0.003927,-0.074603,0.065663,-0.083798,-0.563819,0.002346,4.480510e-16,0.0


In [6]:
km = KMeans(n_clusters = 3,
            random_state = 0)

km.fit(model)
label = km.fit_predict(model)

print('label:', label)
print('Number of iterations:', km.n_iter_)
print('Number of features:', km.n_features_in_)
print('Number of clusters:', km.n_clusters)
print('Inertia:', km.inertia_)

label: [0 0 0 ... 1 1 1]
Number of iterations: 4
Number of features: 20
Number of clusters: 3
Inertia: 2732515.299203973


In [7]:
centroids = km.cluster_centers_
print("Predicted clusters to points: ", label[:10], '\n')
print('Location of centroids:', '\n')
print(centroids)

Predicted clusters to points:  [0 0 0 0 1 1 1 1 1 1] 

Location of centroids: 

[[-5.71622125e-01  3.25787523e+00 -4.57133166e-02  1.68073463e-01
  -1.64269915e-01 -2.86230043e-02  4.94355163e-02 -3.54164911e-02
   2.24159779e-02  3.60776857e-02  1.92136386e-02  1.11621609e-01
   7.46709935e-03 -7.61499949e-04 -6.09226347e-02  7.60651287e-03
  -5.98503537e-03 -3.68786439e-05  1.81259528e-16  0.00000000e+00]
 [-4.61052941e-01 -1.37438752e+00 -1.11800769e-01 -4.99952401e-03
   7.35097146e-02  1.36584889e-02  4.64439081e-02  2.62338208e-03
   2.18233372e-02 -1.94594342e-02  7.88804879e-03 -4.65191344e-02
   6.07283069e-03  5.82074704e-03  2.51308397e-02  4.29038669e-03
   2.44744165e-03 -1.18792792e-03  1.00920319e-16  0.00000000e+00]
 [ 7.30076752e+00  1.20674714e-01  1.36515405e+00 -6.80974308e-01
  -4.99806915e-02 -1.75706454e-02 -6.99905121e-01  1.27125762e-01
  -3.25329066e-01  4.55419287e-02 -1.66056004e-01 -1.81152687e-03
  -9.58922828e-02 -5.73721635e-02  3.69060967e-03 -7.7914603

### Labels

0. LHP group S
1. RHP 3 /4 arm slot
2. RHP overhand maybe slight cut

In [8]:
X['label'] = label
model['label'] = label

#model.groupby(by = 'label').mean().T
X.groupby(by = 'label').mean().T

label,0,1,2
pitch_type,1.000000,1.000000,1.000000
release_speed,92.796611,94.090898,93.591291
release_spin_rate,2250.044138,2284.345249,2259.150321
spin_axis,146.332484,212.188348,194.524472
pfx_-x,-7.494370,7.144355,3.222441
pfx_z,16.196761,16.102812,15.672923
release_extension,6.294994,6.398745,6.362661
release_pos_x,1.893629,-1.782983,-0.780769
release_pos_z,5.985933,5.885456,5.929658
p_throws_L,0.999913,0.000006,0.268393
